In [ ]:
input_dir = "../input"

In [ ]:
# add project root to module path if not added already
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path and os.path.basename(module_path) == "t212-tax-and-profit":
  sys.path.append(module_path)
  print(f"Path '{module_path}' added to module path.")


In [ ]:
from services.csv_consolidator import consolidate

events = consolidate(input_dir)

In [ ]:
from services.event_emitter import EventEmitter
from services.event_processors import DepositWithdrawalProcessor, DividendProcessor, ProfitLossProcessor
from models.input import Action

event_emitter: EventEmitter = EventEmitter()

deposit_withdrawal_processor = DepositWithdrawalProcessor()
event_emitter.subscribe(deposit_withdrawal_processor, [Action.DEPOSIT, Action.WITHDRAWAL])

dividend_processor = DividendProcessor()
event_emitter.subscribe(dividend_processor, Action.DIVIDEND)

profit_loss_processor = ProfitLossProcessor()
event_emitter.subscribe(profit_loss_processor, [Action.BUY, Action.SELL])

event_emitter.start(events)

In [ ]:
import plotly.graph_objects as go

def print_summary(name: str, f, it):
    total = sum([f(x) for x in it])
    print(f"Total {name}: {total:.2f}")

def show_table(headers: list[str], values: list[list]):
    fig = go.Figure(data=[go.Table(
        header=dict(values=headers, align='left'),
        cells=dict(values=values, align='left'))
    ])

    fig.update_layout(width=900, height=1200)
    fig.show()

### Profit calculation

In [ ]:
profit_losses: list[ProfitLossProcessor.ProfitLoss] = profit_loss_processor.result()

In [ ]:
print_summary("profit/loss", lambda p: p.profitloss, profit_losses)
print_summary("shares remaining", lambda p: p.remaining_shares, profit_losses)
print_summary("remaining invested amount", lambda p: p.remaining_invested_amount, profit_losses)

In [ ]:
headers = ['Name', 'Ticker', 'Profit/Loss', 'Remaining shares', 'Remaining invested amount']
values = [
    [p.stock.name for p in profit_losses],
    [p.stock.ticker for p in profit_losses],
    [p.profitloss for p in profit_losses],
    [p.remaining_shares for p in profit_losses],
    [p.remaining_invested_amount for p in profit_losses],
]

show_table(headers, values)

### Dividend calculation

In [ ]:
dividends: list[DividendProcessor.Dividend] = dividend_processor.result()

In [ ]:
print_summary("dividends", lambda d: d.dividend, dividends)
print_summary("withholding taxes", lambda d: d.withholding_tax, dividends)

In [ ]:
headers = ['Name', 'Ticker', 'Dividend', 'Tax', 'Tax currency']
values = [
    [d.stock.name for d in dividends],
    [d.stock.ticker for d in dividends],
    [d.dividend for d in dividends],
    [d.withholding_tax for d in dividends],
    [d.withholding_tax_currency for d in dividends],
]

show_table(headers, values)

### Total gains

### Total deposits and withdrawals

In [ ]:
deposit_withdrawal_result: DepositWithdrawalProcessor.Result = deposit_withdrawal_processor.result()

print(f"Total deposit: {deposit_withdrawal_result.deposit:.2f}")
print(f"Total deposit fees: {deposit_withdrawal_result.deposit_fee}")
print(f"Total withdrawals: {deposit_withdrawal_result.withdrawal:.2f}")